In [12]:
import re,os
import json
import ast,nltk
import datetime
from stemming.porter2 import stem
from collections import defaultdict
from string import punctuation
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import logging
import numpy as np
import time
from gensim.models import Word2Vec,KeyedVectors

In [13]:
training_data = []
# Articles that has category given
with open('articles.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        training_data.append(line)
        line = data_file.readline()
    data_file.close()

In [61]:
testing_data = []
# Articles that has no category given
with open('articles_wo.json','r') as data_file:
    line = data_file.readline()
    while line:
        line = ast.literal_eval(line)
        testing_data.append(line)
        line = data_file.readline()
    data_file.close()

In [14]:
print len(training_data)

10369


In [15]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    text = re.sub("[^a-zA-Z]"," ",sentence)
    words = text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [16]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    return sentences

In [55]:
words = []
classes = []
documents = []
no_classes = []
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    words.extend(nltk.word_tokenize(pattern['desc']))
    documents += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(pattern['desc'], tokenizer)]]
    classes.append(pattern['category'])
    if pattern['category'] not in no_classes:
        no_classes.append(pattern['category'])

In [62]:
test_documents = []
for pattern in testing_data:
    test_documents += [[stem(str(word)) for word in sentence] \
                          for sentence in [text_to_sentences(pattern['desc'],tokenizer)]]

In [41]:
output = []
# create an empty array for our output
output_empty = [0] * len(no_classes)
for Class in classes:
    output_row = list(output_empty)
    output_row[no_classes.index(Class)] = 1
    output.append(output_row)

In [58]:
stops = set(stopwords.words("english"))
words = [stem(w.lower()) for w in words if w not in stops]
words = list(set(words))

In [59]:
print len(words)

80053


In [107]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 1  # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 4           # Context window size                                                                                    
downsampling = 1e-5   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_1minwords_4context"
model.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-10-29 18:57:59,931 : INFO : collecting all words and their counts
2017-10-29 18:57:59,933 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-29 18:58:00,106 : INFO : PROGRESS: at sentence #10000, processed 204500 words, keeping 182852 word types
2017-10-29 18:58:00,116 : INFO : collected 188628 word types from a corpus of 211322 raw words and 10369 sentences
2017-10-29 18:58:00,117 : INFO : Loading a fresh vocabulary


Training model...


2017-10-29 18:58:01,191 : INFO : min_count=1 retains 188628 unique words (100% of original 188628, drops 0)
2017-10-29 18:58:01,192 : INFO : min_count=1 leaves 211322 word corpus (100% of original 211322, drops 0)
2017-10-29 18:58:01,686 : INFO : deleting the raw counts dictionary of 188628 items
2017-10-29 18:58:01,692 : INFO : sample=0.001 downsamples 1 most-common words
2017-10-29 18:58:01,696 : INFO : downsampling leaves estimated 210615 word corpus (99.7% of prior 211322)
2017-10-29 18:58:01,702 : INFO : estimated required memory for 188628 words and 100 dimensions: 245216400 bytes
2017-10-29 18:58:02,222 : INFO : resetting layer weights
2017-10-29 18:58:04,685 : INFO : training model with 4 workers on 188628 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=4
2017-10-29 18:58:04,686 : INFO : expecting 10369 sentences, matching count from corpus used for vocabulary survey
2017-10-29 18:58:05,693 : INFO : PROGRESS: at 54.89% examples, 576417 words/s, in_qs

In [108]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [109]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [110]:
DataVecs = getAvgFeatureVecs( documents, model, num_features )

Review 0 of 10369
Review 1000 of 10369
Review 2000 of 10369
Review 3000 of 10369
Review 4000 of 10369
Review 5000 of 10369
Review 6000 of 10369
Review 7000 of 10369
Review 8000 of 10369
Review 9000 of 10369
Review 10000 of 10369


In [111]:
print DataVecs[0]

[ 0.00017184154  2.4503889e-05  0.00012210625 -0.00015397034 -0.0015202732
  0.00028750111  0.0012024901  0.0011574512 -0.0013270631 -0.00064087883
  0.00028746979  0.0001006107 -0.00079584038  0.0015622324  8.0731538e-05
  0.0010361368  0.00036334324  0.00018084297 -0.0014883342 -0.0019142148
  0.0010670282  1.2763992e-06  5.8317633e-05  7.0355972e-05 -0.0010568126
 -0.00021687347 -0.00106138 -0.00053426733 -0.0014644218 -0.0017388701
  0.00020616575  0.0011127155 -0.0005813003  0.00012722127  0.00088584367
  0.00097818095 -0.00044892763  0.00040844048 -0.0010613813  0.00087935478
  0.0009803304 -0.00076347655  0.0014898952 -0.00033581083  0.0007813529
 -9.2976384e-05  0.00032211664  0.00072148183  0.00043570729 -0.00040095301
  0.00023414203  0.00069072985  7.26954e-05 -0.00051793239 -2.1585842e-05
 -0.00081479376 -0.00016346795  6.9226186e-05  0.00047237841 -0.00029495278
  0.00051753635  0.00036791475  0.0020165471  0.00083393433  0.0011918705
 -0.00038207743 -9.9803669e-05  0.0003

In [112]:

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(x, show_details=False):
    #x = bow(sentence.lower(), words, show_details)
    #if show_details:
    #    print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [113]:
# ANN and Gradient Descent code from https://iamtrask.github.io//2015/07/27/python-network-part2/
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(no_classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(no_classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': no_classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [114]:
X = np.array(DataVecs)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=10, alpha=0.1, epochs=1000, dropout=False, dropout_percent=0.1)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 10 neurons, alpha:0.1, dropout:False 
Input matrix: 10369x100    Output matrix: 1x20
('saved synapses to:', 'synapses.json')
('processing time:', 146.79171299934387, 'seconds')


In [115]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 1  # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 4           # Context window size                                                                                    
downsampling = 1e-5   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
testmodel = Word2Vec(test_documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
testmodel.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100testfeatures_1minwords_4context"
testmodel.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-10-29 19:16:06,620 : INFO : collecting all words and their counts
2017-10-29 19:16:06,621 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-29 19:16:06,625 : INFO : collected 1903 word types from a corpus of 2073 raw words and 483 sentences
2017-10-29 19:16:06,627 : INFO : Loading a fresh vocabulary
2017-10-29 19:16:06,634 : INFO : min_count=1 retains 1903 unique words (100% of original 1903, drops 0)
2017-10-29 19:16:06,636 : INFO : min_count=1 leaves 2073 word corpus (100% of original 2073, drops 0)
2017-10-29 19:16:06,644 : INFO : deleting the raw counts dictionary of 1903 items
2017-10-29 19:16:06,650 : INFO : sample=0.001 downsamples 6 most-common words
2017-10-29 19:16:06,652 : INFO : downsampling leaves estimated 2018 word corpus (97.4% of prior 2073)
2017-10-29 19:16:06,654 : INFO : estimated required memory for 1903 words and 100 dimensions: 2473900 bytes
2017-10-29 19:16:06,660 : INFO : resetting layer weights
2017-10-29 19:16:06,687 : IN

Training model...


In [116]:
TestDataVecs = getAvgFeatureVecs( test_documents, testmodel, num_features )

Review 0 of 483


In [117]:
print TestDataVecs[0]

[ 0.0006890658 -0.00076296009 -0.0029413692  0.0013572672  0.0037287914
 -0.0013011911 -5.1862997e-05 -0.00037936732  0.0011542493  0.00092733761
  0.00039084792  0.00098856304 -0.00011817757  0.0015670626  0.00030479493
  0.00073972395 -0.0013632241 -0.00051237282  0.00082528099 -0.0004652932
 -0.00096308258  0.0010061627 -0.00016638165 -0.0013457348 -0.0023903108
  0.0015798633 -0.00073414524  0.002382225  0.0020401925 -0.00055952312
  0.001046675 -0.00058265467 -0.00016036895  0.0012069821 -0.0027204474
 -0.00081853732 -0.0011815737  0.0026908009  0.00046523407 -0.0017699723
 -0.0024348643 -0.00051858962 -0.0017020455  0.0016795258  0.00082494505
 -0.00040003934 -0.00031471313  0.0012815975 -0.0011005206 -0.0010705766
  0.0010046149 -0.0012450011  0.00061226469 -0.0017370856 -7.8674566e-06
  0.00047870026  0.0014340116  0.001262779  0.00072022204 -0.00025013371
 -0.0015941704 -0.00015020691  0.00030866565 -0.00082557416 -0.00013252324
  0.00018200482 -0.00066850494  0.00026254755  0

In [118]:
# probability threshold
ERROR_THRESHOLD = 0.01
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])



In [124]:
def classify(vec, show_details=False):
    results = think(vec, show_details)
    print results
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[no_classes[r[0]],r[1]] for r in results]
    #print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

In [123]:
print classes[0]

politics


In [125]:
res = classify(TestDataVecs[0])
print res

[ 1.0  7.4535529e-128  1.4285547e-46  9.3041586e-151  4.4277408e-144
  8.6185771e-120  3.8687555e-127  4.854048e-140  5.2323548e-163
  2.2749179e-137  2.2924973e-62  4.0771911e-155  1.2615845e-137
  7.3228435e-52  1.619236e-165  1.5891255e-97  6.3797803e-56
  2.1281178e-155  4.9241489e-164  6.4979743e-126]
[['politics', 0.99999999999999999946]]
